## Connecting to Amazon Keyspaces using SigV4 authentication plugin for temporary credentials.  
#####   This plugin enables IAM users, roles, and federated identities to add authentication information to Amazon Keyspaces API requests using the AWS Signature Version 4 process (SigV4).

More info about [SigV4](https://docs.aws.amazon.com/general/latest/gr/signature-version-4.html) 


### Requrements 

The Notebook execution role must include permissions to access Amazon Keyspaces and Assume the role. 


*  To access Amazon Keyspaces database - can use AmazonKeyspacesReadOnlyAccess or AmazonKeyspacesFullAccess managed policies. See [AWS Identity and Access Management for Amazon Keyspaces](https://docs.aws.amazon.com/keyspaces/latest/devguide/security-iam.html). Use the least privilege approach for your production application.

* To assume the role you need to have [sts:AssumeRole action](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html) permissions
    ```
    {
      "Version": "2012-10-17",  
      "Statement": [  
        {  
           "Action": [  
           "sts:AssumeRole"  
          ],  
          "Effect": "Allow",  
          "Resource": "*"  
        }
      ]
    }
    ```

#### Note:
Amazon Keyspaces is available in the following [AWS Regions](https://docs.aws.amazon.com/keyspaces/latest/devguide/programmatic.endpoints.html)

This notebook was tested with conda_python3 kernel and should work with Python 3.x.

In [ ]:
# Installing Cassanda SigV4 
%pip install  cassandra-sigv4

In [ ]:
# Get Security certificate 
!curl https://certs.secureserver.net/repository/sf-class2-root.crt -O 


In [ ]:
# Import 
from sagemaker import get_execution_role
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2 , CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

In [ ]:
#Getting credentials from the role
client = boto3.client('sts')

#Get notebook Role
role = get_execution_role()
role_info = {
    'RoleArn':role,
    'RoleSessionName': 'session1'
}
print(role_info)

credentials = client.assume_role(**role_info)


In [ ]:
# Connecting to Cassandra using temporarly credentials. 
session = boto3.session.Session(
    aws_access_key_id=credentials['Credentials']['AccessKeyId'],
    aws_secret_access_key=credentials['Credentials']['SecretAccessKey'],
    aws_session_token=credentials['Credentials']['SessionToken']
)

region_name = session.region_name

#print(region_name)
#print(credentials['Credentials']['AccessKeyId'])
#print(credentials['Credentials']['SecretAccessKey'])
#print(credentials['Credentials']['SessionToken'])

#Set Context
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('sf-class2-root.crt')
ssl_context.verify_mode = CERT_REQUIRED

auth_provider = SigV4AuthProvider(session)

keyspaces_host = 'cassandra.' + region_name + '.amazonaws.com'

cluster = Cluster([keyspaces_host], ssl_context=ssl_context, auth_provider=auth_provider,   port=9142)
session = cluster.connect()


# Read data from Keyspaces system table.  This table   
r = session.execute('select * from system_schema.keyspaces')
print(r.current_rows)

In [ ]:
# Read Keyspaces row into Panda DataFrame 
from pandas import DataFrame
df = DataFrame(r)
print(df)

In [ ]:
####
## Read your own tables. 
####

#keyspaces = 'your_schema'
#table ='your_table'

#rows = session.execute('select * from '+ keyspaces + '.' + table )
#df = DataFrame(rows)
#print(df)

## The end.